### Simple AR Program
1) Compute point correspondences (2D and AR tag) <br>
2) Estimate the pose of the camera <br>
3) Project 3D content to the image plane

In [35]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import glob
import skimage.io as io

**Inputting the video frames**

In [36]:
FRAMES_PATH = "data/apriltagims/*"

video_frames = glob.glob(FRAMES_PATH)
video_frame.sort()

In [37]:
initial_points = 1.0e+02 * np.array([[1.981631469726562,3.165294189453125],
                                    [3.786268920898438,3.424402770996094],
                                    [4.036800842285157,1.686005859375000],
                                    [2.333528289794922,1.491907043457031]])

K = 1.0e+02 * np.array([[7.661088867187500, 0, 3.139585628047498],
                        [0, 7.699354248046875,  2.503607131410900],
                        [0 ,                  0,   0.010000000000000 ]])

tag_width = 0.13;
tag_height = 0.13;
cube_depth = 0.13;

corner_pts = np.array([[  tag_width/2,  tag_height/2],
               [-tag_width/2,  tag_height/2],
               [-tag_width/2, -tag_height/2],
               [tag_width/2, -tag_height/2]])

**Implement KLT Tracking**

In [38]:
def track_corners(video_imgs, initial_pts):
    pass

In [39]:
corners = track_corners(video_frames, initial_points)

**Homography Estimation**

In [40]:
from utils import vectorize, solve_homography

**Pose Estimation (from Homography) of co-planar points**

In [41]:
def ar_cube(H, K, proj_points):
    '''
    function for pose prediction of the camera, returns the
    extrinsic of the camera given the coplanar points and intrisic 
    
    :param H(np.array): size(3x3) homography matrix
    :param K(np.array): size(3x3) intrinsics of camera
    :param proj_points: size (N x 2) matrix of the projected points in pixel coordinates
    :Return t: size (3 x 1) vector of the translation of the transformation
    :Return R: size (3 x 3) matrix of the rotation of the transformation (orthogonal matrix)
    '''
    
    pass
    

In [42]:
def draw_ar_cube(points, im):
    '''
    Uses projection equation to create images from points
    
    :param points(np.array): size(2x8) : projected points to connect
    :param im(np.array): size (nxm) to render with the image
    
    Returns
    im_out - (np.array): size(nxm) with the cube drawn on the image
    '''
    
    pass

**Projecting Points**